In [3]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import matplotlib
matplotlib.use("Agg")

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks.callbacks import ModelCheckpoint
from keras.layers.pooling import AveragePooling2D, MaxPooling2D
from keras.applications.mobilenet_v2 import MobileNetV2
import tensorflow as tf
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input, Conv2D, BatchNormalization
from keras.models import Model
from keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 
from google.colab.patches import cv2_imshow
from imutils import paths
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import argparse
import pickle
import cv2
import os   
from sklearn.utils import shuffle
from keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip -q '/content/gdrive/My Drive/Data Science/Datasets/AntiFire Dataset Large.zip'

In [ ]:
resnet = load_model('/content/gdrive/My Drive/Data Science/Models/final/ResNet50')
inception = load_model('/content/gdrive/My Drive/Data Science/Models/final/Inceptionv3')
mobilenet = load_model('/content/gdrive/My Drive/Data Science/Models/final/MobileNetv2')

In [ ]:
test_datagen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True)

test_generator = test_datagen.flow_from_directory(
    'AntiFire Dataset/test/', target_size=(224, 224),
    batch_size=32, class_mode='binary', shuffle=False)

y_test = test_generator.labels

In [ ]:
predictions_resnet = resnet.predict_generator(test_generator, steps=np.ceil(3343 / 32), verbose=1)
predictions_inception = inception.predict_generator(test_generator, steps=np.ceil(3343 / 32), verbose=1)
predictions_mobilenet = mobilenet.predict_generator(test_generator, steps=np.ceil(3343 / 32), verbose=1)

### Precision-recall

In [ ]:
resnet_prec, resnet_rec, _ = precision_recall_curve(y_test, predictions_resnet)
inception_prec, inception_rec, _ = precision_recall_curve(y_test, predictions_inception)
mobilenet_prec, mobilenet_rec, _ = precision_recall_curve(y_test, predictions_mobilenet)

plt.figure(figsize=(15, 9))
plt.grid()
plt.title('Precision-recall curves')

plt.step(resnet_rec, resnet_prec, label='ResNet50')
plt.step(inception_rec, inception_prec, label='Inception v3')
plt.step(mobilenet_rec, mobilenet_prec, label='MobileNet v2')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.xticks(np.arange(0.0, 1.00001, 0.1))
plt.yticks(np.arange(0.2, 1.00001, 0.1))
plt.legend()
plt.show()

### ROC

In [ ]:
resnet_fpr, resnet_tpr, _ = roc_curve(y_test, predictions_resnet)
inception_fpr, inception_tpr, _ = roc_curve(y_test, predictions_inception)
mobilenet_fpr, mobilenet_tpr, _ = roc_curve(y_test, predictions_mobilenet)

plt.figure(figsize=(15, 9))
plt.grid()
plt.title('ROC curves')

plt.step(resnet_fpr, resnet_tpr, label='ResNet50')
plt.step(inception_fpr, inception_tpr, label='Inception v3')
plt.step(mobilenet_fpr, mobilenet_tpr, label='MobileNet v2')

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.xticks(np.arange(0.0, 1.00001, 0.1))
plt.yticks(np.arange(0.2, 1.00001, 0.1))
plt.legend()
plt.show()